In [ ]:
# IMPORT AUPassata Regular + parse config

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import yaml

font_path = "./AUPassata_Rg.ttf"
fm.fontManager.addfont(font_path)


def init_plotting():
    plt.rcParams['figure.figsize'] = (8, 3)
    plt.rcParams['font.size'] = 10
    plt.rcParams['font.family'] = 'AU Passata'
    plt.rcParams['axes.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['axes.titlesize'] = 1.5 * plt.rcParams['font.size']
    plt.rcParams['legend.fontsize'] = plt.rcParams['font.size']
    plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['savefig.dpi'] = 200
    plt.rcParams['xtick.major.size'] = 3
    plt.rcParams['xtick.major.width'] = 1
    plt.rcParams['ytick.major.size'] = 3
    plt.rcParams['ytick.major.width'] = 1
    plt.rcParams['legend.frameon'] = False
    plt.rcParams['axes.linewidth'] = 1

    ax = plt.gca()
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    
    
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
INPUT_FRAGS_DIR = config['input_frags_dir']
INPUT_DHS_DIR = config['input_dhs_dir']
SORTED_FRAGS_DIR = config['sorted_frags_dir']
SORTED_DHS_DIR = config['sorted_dhs_dir']
OUTPUT_DIR = config['output_dir']
MATRIX_COLUMNS = config['matrix_columns']

In [ ]:
# Get healhty sample coverage

import glob
import subprocess


healhty_files = glob.glob(f'{INPUT_FRAGS_DIR}*.hg38.frag.gz')

coverages = []
for f in healhty_files:
    coverages.append(int(subprocess.check_output(['wc', '-l', f]).split()[0]))
    
init_plotting()
mn, mx = min(coverages), max(coverages)
plt.hist(
    coverages,
    bins=30,
    edgecolor='black',
    alpha=0.8
)

plt.plot([], [], ' ', label=f"Min coverage = {mn:,}")
plt.plot([], [], ' ', label=f"Max coverage = {mx:,}")

plt.legend(loc='upper right')
plt.xlabel('Coverage')
plt.ylabel('Frequency')
plt.title(f'Coverage distribution\n({len(coverages)} healthy samples)')

plt.tight_layout()
plt.show()

In [ ]:
# Get healhty sample coverage

import glob
import subprocess


healhty_files = glob.glob(f'{SORTED_FRAGS_DIR}*.hg38.frag.gz')

coverages = []
for f in healhty_files:
    coverages.append(int(subprocess.check_output(['wc', '-l', f]).split()[0]))
    
init_plotting()
mn, mx = min(coverages), max(coverages)
plt.hist(
    coverages,
    bins=30,
    edgecolor='black',
    alpha=0.8
)

plt.plot([], [], ' ', label=f"Min coverage = {mn:,}")
plt.plot([], [], ' ', label=f"Max coverage = {mx:,}")

plt.legend(loc='upper right')
plt.xlabel('Coverage')
plt.ylabel('Frequency')
plt.title(f'Coverage distribution\n({len(coverages)} sorted healthy samples)')

plt.tight_layout()
plt.show()

In [ ]:
# Get Lymphoid and Lymphoid_negative DHSs "coverage" (number of DHS sites)

import glob
import subprocess
import re
from collections import defaultdict
import numpy as np

initial_dhs_files = sorted(glob.glob(f'{SORTED_DHS_DIR}*_sorted.bed'))
preprocessed_dhs_files = sorted(glob.glob(f'{SORTED_DHS_DIR}*_sorted_wl{MATRIX_COLUMNS}.bed'))
downsampled_dhs_files = sorted(glob.glob(f'{SORTED_DHS_DIR}*_sorted_wl{MATRIX_COLUMNS}_downsampled.bed'))

def clean_tag(name: str) -> str:
    return re.sub(r'_sorted(_wl\d+)?(_downsampled)?\.bed$', '', name)

groups = ['initial', 'preprocessed', 'downsampled']
coverages = defaultdict(list)
for f1, f2, f3 in zip(initial_dhs_files, preprocessed_dhs_files, downsampled_dhs_files):
    tag1, tag2, tag3 = clean_tag(f1.rsplit('/', 1)[1]), clean_tag(f2.rsplit('/', 1)[1]), clean_tag(f3.rsplit('/', 1)[1])
    cov1 = int(subprocess.check_output(['wc', '-l', f1]).split()[0])
    coverages[tag1].append(cov1)
    cov2 = int(subprocess.check_output(['wc', '-l', f2]).split()[0])
    coverages[tag2].append(cov2)
    cov3 = int(subprocess.check_output(['wc', '-l', f3]).split()[0])
    coverages[tag3].append(cov3)
    

groups, coverages
x = np.arange(len(groups))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

init_plotting()

for tag, coverage in coverages.items():
    offset = width * multiplier
    rects = plt.bar(x + offset, coverage, width, label=tag)
    plt.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
plt.xlabel('Phases')
plt.ylabel('Coverage')
plt.xticks(x + (width/2), groups)
plt.legend(loc='best', )

plt.tight_layout()
plt.show()

In [ ]:
# Show output matrixes coverage (without downsampling)

import glob
import subprocess
import re
import numpy as np
from collections import defaultdict

healhty_files = glob.glob(f'{SORTED_FRAGS_DIR}*.hg38.frag.gz')

coverages = defaultdict(int)
regex = r'EE\d+'

for f in healhty_files:
    sample = re.search(regex, f)[0]
    coverages[sample] = int(subprocess.check_output(['wc', '-l', f]).split()[0])

def clean_tag(name: str) -> str:
    m = re.search(r'__([^_.]+(?:_negative)?)\.npy$', name)
    return m.group(1) if m else name

output_files = glob.glob(f'../data/cristiano_healhty_without_downsampling/*npy')

output_coverages = defaultdict(list)
for f in output_files:
    cov = np.load(f).sum()
    output_coverages[clean_tag(f)].append(cov)
    
fig, axes = plt.subplots(
    3,
    1,
    sharex=True,
    sharey=True,
    figsize=(8, 6),
)

mn, mx = min(coverages.values()), max(coverages.values())
axes[0].hist(
    coverages.values(),
    bins=50,
    edgecolor="black",
    alpha=0.8
)
axes[0].plot([], [], ' ', label=f"Min coverage = {mn:,}")
axes[0].plot([], [], ' ', label=f"Max coverage = {mx:,}")

axes[0].legend(loc='upper right')

axes[0].set_ylabel("Frequency")
axes[0].set_title("Frags coverage distribution")


mn, mx = int(min(output_coverages["Lymphoid"])), int(max(output_coverages["Lymphoid"]))
axes[1].hist(
    output_coverages["Lymphoid"],
    bins=50,
    edgecolor="black",
    alpha=0.8
)
axes[1].plot([], [], ' ', label=f"Min coverage = {mn:,}")
axes[1].plot([], [], ' ', label=f"Max coverage = {mx:,}")

axes[1].legend(loc='upper right')
axes[1].set_ylabel("Frequency")
axes[1].set_title("Lymphoid matrix coverage distribution")

mn, mx = int(min(output_coverages["Lymphoid_negative"])), int(max(output_coverages["Lymphoid_negative"]))
axes[2].hist(
    output_coverages["Lymphoid_negative"],
    bins=50,
    edgecolor="black",
    alpha=0.8
)
axes[2].plot([], [], ' ', label=f"Min coverage = {mn:,}")
axes[2].plot([], [], ' ', label=f"Max coverage = {mx:,}")

axes[2].legend(loc='upper right')
axes[2].set_xlabel("Coverage")
axes[2].set_ylabel("Frequency")
axes[2].set_title("Lymphoid_negative matrix coverage distribution")

plt.tight_layout()
plt.show()

In [ ]:
# Show output matrixes coverage (with downsampling)

import glob
import subprocess
import re
import numpy as np
from collections import defaultdict

healhty_files = glob.glob(f'{SORTED_FRAGS_DIR}*.hg38.frag.gz')

coverages = defaultdict(int)
regex = r'EE\d+'

for f in healhty_files:
    sample = re.search(regex, f)[0]
    coverages[sample] = int(subprocess.check_output(['wc', '-l', f]).split()[0])

def clean_tag(name: str) -> str:
    m = re.search(r'__([^_.]+(?:_negative)?)\.npy$', name)
    return m.group(1) if m else name

output_files = glob.glob(f'{OUTPUT_DIR}*.npy')

output_coverages = defaultdict(list)
for f in output_files:
    cov = np.load(f).sum()
    output_coverages[clean_tag(f)].append(cov)
    
fig, axes = plt.subplots(
    3,
    1,
    sharex=True,
    sharey=True,
    figsize=(8, 6),
)

mn, mx = min(coverages.values()), max(coverages.values())
axes[0].hist(
    coverages.values(),
    bins=50,
    edgecolor="black",
    alpha=0.8
)
axes[0].plot([], [], ' ', label=f"Min coverage = {mn:,}")
axes[0].plot([], [], ' ', label=f"Max coverage = {mx:,}")

axes[0].legend(loc='upper right')

axes[0].set_ylabel("Frequency")
axes[0].set_title("Frags coverage distribution")


mn, mx = int(min(output_coverages["Lymphoid"])), int(max(output_coverages["Lymphoid"]))
axes[1].hist(
    output_coverages["Lymphoid"],
    bins=50,
    edgecolor="black",
    alpha=0.8
)
axes[1].plot([], [], ' ', label=f"Min coverage = {mn:,}")
axes[1].plot([], [], ' ', label=f"Max coverage = {mx:,}")

axes[1].legend(loc='upper right')
axes[1].set_ylabel("Frequency")
axes[1].set_title("Lymphoid matrix coverage distribution")

mn, mx = int(min(output_coverages["Lymphoid_negative"])), int(max(output_coverages["Lymphoid_negative"]))
axes[2].hist(
    output_coverages["Lymphoid_negative"],
    bins=50,
    edgecolor="black",
    alpha=0.8
)
axes[2].plot([], [], ' ', label=f"Min coverage = {mn:,}")
axes[2].plot([], [], ' ', label=f"Max coverage = {mx:,}")

axes[2].legend(loc='upper right')
axes[2].set_xlabel("Coverage")
axes[2].set_ylabel("Frequency")
axes[2].set_title("Lymphoid_negative matrix coverage distribution")

plt.tight_layout()
plt.show()

In [ ]:
# Show global patterns for both Lymphoid and Lymphoid_negative DHS

import glob
import numpy as np
from collections import defaultdict
from functools import partial


output_files = glob.glob(f'{OUTPUT_DIR}*.npy')
shape = (300, 2500)

sum_signal = defaultdict(partial(np.zeros, shape))
for f in output_files:
    matrix = np.load(f)
    sum_signal[clean_tag(f)] += matrix

dhs_pos = MATRIX_COLUMNS // 2
relative_midpoints = np.sum(sum_signal['Lymphoid'], axis=0) # (2500,)
frag_lengths = np.sum(sum_signal['Lymphoid'], axis=1) # (300,)

fig, axes = plt.subplots(2, 2, figsize=(10, 6))
fig.suptitle('Summary of global patterns using downsampled fragments (cov=199,000) and both the Lymphoid and Lymphoid_negative DHSs', fontsize=16)

lengths = np.arange(len(frag_lengths))
axes[0, 0].bar(lengths, frag_lengths, width=1)
axes[0, 0].set_title('Fragment length distribution\n(Lymphoid)')
axes[0, 0].set_xlabel('Fragment length')
axes[0, 0].set_ylabel('Count')

x = np.arange(len(relative_midpoints))
axes[0, 1].plot(x, relative_midpoints)
axes[0, 1].axvline(x=dhs_pos, color='red', linestyle='--', linewidth=2, label='DHS site')
axes[0, 1].axvline(x=dhs_pos-1000, color='green', linestyle='--', linewidth=2, label='Lower threshold')
axes[0, 1].axvline(x=dhs_pos+1000, color='green', linestyle='--', linewidth=2, label='Upper threshold')
axes[0, 1].set_title('Relative midpoint coverage\n(Lymphoid)')
axes[0, 1].set_xlabel('Relative position')
axes[0, 1].set_ylabel('Count')

relative_midpoints = np.sum(sum_signal['Lymphoid_negative'], axis=0) # (2500,)
frag_lengths = np.sum(sum_signal['Lymphoid_negative'], axis=1) # (300,)

lengths = np.arange(len(frag_lengths))
axes[1, 0].bar(lengths, frag_lengths, width=1)
axes[1, 0].set_title('Fragment length distribution\n(Lymphoid_negative)')
axes[1, 0].set_xlabel('Fragment length')
axes[1, 0].set_ylabel('Count')

x = np.arange(len(relative_midpoints))
axes[1, 1].plot(x, relative_midpoints)
axes[1, 1].axvline(x=dhs_pos, color='red', linestyle='--', linewidth=2, label='DHS site')
axes[1, 1].axvline(x=dhs_pos-1000, color='green', linestyle='--', linewidth=2, label='Lower threshold')
axes[1, 1].axvline(x=dhs_pos+1000, color='green', linestyle='--', linewidth=2, label='Upper threshold')
axes[1, 1].set_title('Relative midpoint coverage\n(Lymphoid_negative)')
axes[1, 1].set_xlabel('Relative position')
axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()